In [190]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import re
import time
import random
import pandas as pd
from multiprocessing.dummy import Pool

In [191]:
#淘宝和天猫的 搜索页面
init_searchurl_taobao ='https://s.taobao.com/search?'
#淘宝和天猫的商品链接
init_itemurl_tmall = 'https://detail.tmall.com/item.htm?id={}'
init_itemurl_taobao = 'https://item.taobao.com/item.htm?id={}'

headers={
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'zh-CN,zh;q=0.9',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.62 Safari/537.36'
        }
#淘宝搜索页面
init_searchurl_taobao ='https://s.taobao.com/search?'


init_commentUrl_tmall = 'https://dsr-rate.tmall.com/list_dsr_info.htm?itemId={}'
init_commentUrl_taobao = 'https://rate.taobao.com/detailCommon.htm?auctionNumId={}'

In [192]:
def getitemurls_taobao(target,sort='sale-desc',maxpage=1):
    itemurls = {}
    itemurl_taobao = []
    itemurl_tmall = []
    for page in range(0,maxpage):
        params = {
                    'q' : target,
                    'sort' : sort,
                    's' : str(44*(page))
        }
        res = requests.get(init_searchurl_taobao,params=params,headers=headers)
        ids = re.findall('"nid":"(.*?)","category"',res.text,re.S)
        urls = re.findall('"detail_url":"//(.*?)","view_price"',res.text,re.S)
#        print(len(ids))
#        print(len(urls))
        if len(ids) == 0:
            print('被干了这次不算')
        info = dict(ids = ids , urls=urls)
        for i in range(0,len(ids)):
        #由于淘宝页面返回的URL是带//OOOC3这种乱码的，所以url只是看一下到底是淘宝还是天猫，并不作为真正的url传回去
        #返回的URL是用init_itemurl拼出来的
            if 'tmall' in info['urls'][i]:
                itemurl_tmall.append(init_itemurl_tmall.format(info['ids'][i]))
            elif 'taobao' in info['urls'][i]:
                itemurl_taobao.append(init_itemurl_taobao.format(info['ids'][i]))  
#        print("稍等一下")
        time.sleep(random.randint(0,1))
    itemurls['tmall'] = itemurl_tmall
    itemurls['taobao'] = itemurl_taobao
    print('共有商品%d个，其中淘宝%d个，天猫%d个'%((len(itemurl_tmall)+len(itemurl_taobao)),len(itemurl_taobao),len(itemurl_tmall)))
    return itemurls

In [193]:
#获取淘宝的商品信息
def getitemInfo_taobao(itemUrl):
    res = openlink(itemUrl)
    print("获取商品: "+ res.url)
    selector = etree.HTML(res.text)

    title = selector.xpath("//h3[@class='tb-main-title']/@data-title")[0]

    price = selector.xpath("//em[@class = 'tb-rmb-num']/text()")[0] # 淘宝

    itemID = re.findall('id=(\d+)',res.url)[0]

    itemDetail = "".join(selector.xpath('//div[@class="attributes"]/ul/li/text()'))

    shopUrl = selector.xpath('//div[@class="tb-shop-search"]/div/form/@action')[0].lstrip('//').rstrip('search.htm')

    shopId = re.findall('shopId=(.*?);',selector.xpath('//meta[@name="microscope-data"]/@content')[0])[0]

    #获取评论信息 
    res_com = openlink(init_commentUrl_taobao.format(itemID))
    jc = json.loads(res_com.text.lstrip('\r\n(').rstrip(')'))
    #评论数
    totalComment = jc['data']['count']['total']
    #好评数
    good = jc['data']['count']['good']
    #中评数
    normal = jc['data']['count']['normal']
    #差评数
    bad = jc['data']['count']['bad']

    itemInfo = pd.DataFrame(dict(title=title,
                             price=price,
                            itemID=itemID,
                            itemDetail=[itemDetail],
                            itemLink =res.url,
                            shopUrl = shopUrl,
                             shopId  = shopId,
                             totalComment = totalComment,
                             good = good,
                             normal = normal,
                             bad = bad,
                            source = 'taobao'
                            ),
                            columns=['source','itemID','itemLink','title' ,'price','totalComment','good','normal', 'bad','itemDetail','shopId','shopUrl'])
    return itemInfo

In [194]:
# 获取天猫的商品信息
def getitemInfo_tmall(itemUrl):
    res = openlink(itemUrl)
    print("获取商品: "+ res.url)
    selector = etree.HTML(res.text)    
    title = selector.xpath("//div[@class='tb-detail-hd']/h1/text()")[0].replace(' ','').replace('\t','').replace('\r','').replace('\n','')
    price = re.findall('"defaultItemPrice":"(.*?)"',res.text)[0]
    itemID = re.findall('id=(\d+)',res.url)[0]
    itemDetail = "".join(selector.xpath('//div[@class="attributes"]/div/ul/li/text()'))

    shopUrl = selector.xpath('//div[@class="slogo"]/a/@href')[0].lstrip('//')
    shopId = selector.xpath('//div[@id="LineZing"]/@shopid')

    #获取评论信息 

    res_com = openlink(init_commentUrl_tmall.format(itemID))

    jc = json.loads(res_com.text.lstrip('\r\njsonp128(').rstrip(')\n'))

    #评论数
    totalComment = jc['dsr']['rateTotal']

    #评分
    score = jc['dsr']['gradeAvg']

    itemInfo = pd.DataFrame(dict(title=title,
                                 price=price,
                                itemID=itemID,
                                itemDetail=[itemDetail],
                                itemLink =res.url,
                                shopUrl = shopUrl,
                                 shopId  = shopId,
                                 totalComment = totalComment,
                                 score = score,
                                 source = 'tmall'
                                ),columns=['source','itemID','itemLink','title' ,'price','totalComment','score','itemDetail','shopId','shopUrl'])
    return itemInfo

In [195]:
def openlink(link):
    maxTryNum = 10
    for tries in range(0,maxTryNum):
        try:
            res = requests.get(link, headers = headers)
            return res
        except:
            if tries < (maxTryNum - 1):
                print("啊呀")
                continue
            else:
                print("Has tried %d times to access url %s, all failed!", maxNum, link)
                break

In [198]:
target = '行车记录仪'
itemUrl = getitemurls_taobao(target,maxpage=10)

共有商品440个，其中淘宝92个，天猫348个


In [199]:
info_taobao =[]
info_tmall = []

pool = Pool(8)
time3 =time.time()
info_taobao = pool.map(getitemInfo_taobao,itemUrl['taobao'])
info_tmall = pool.map(getitemInfo_tmall,itemUrl['tmall'])
pool.close()
pool.join()

time4 =time.time()
print(str(time4-time3))

获取商品: https://item.taobao.com/item.htm?id=573481101871
获取商品: https://item.taobao.com/item.htm?id=561869766169
获取商品: https://item.taobao.com/item.htm?id=571908069396
获取商品: https://item.taobao.com/item.htm?id=564169946669
获取商品: https://item.taobao.com/item.htm?id=573718312859
获取商品: https://item.taobao.com/item.htm?id=574246632987
获取商品: https://item.taobao.com/item.htm?id=553684920820
获取商品: https://item.taobao.com/item.htm?id=573972480638
获取商品: https://item.taobao.com/item.htm?id=568772214439
获取商品: https://item.taobao.com/item.htm?id=534159083192
获取商品: https://item.taobao.com/item.htm?id=526397484286
获取商品: https://item.taobao.com/item.htm?id=567482957351
获取商品: https://item.taobao.com/item.htm?id=569518376281
获取商品: https://item.taobao.com/item.htm?id=520347883076
获取商品: https://item.taobao.com/item.htm?id=566147524797
获取商品: https://item.taobao.com/item.htm?id=569519448720
获取商品: https://item.taobao.com/item.htm?id=559369594465
获取商品: https://item.taobao.com/item.htm?id=570805785090
获取商品: http

获取商品: https://detail.tmall.com/item.htm?id=568713228093
啊呀
获取商品: https://detail.tmall.com/item.htm?id=574452861350
获取商品: https://detail.tmall.com/item.htm?id=554871690861
获取商品: https://detail.tmall.com/item.htm?id=572212754563
获取商品: https://detail.tmall.com/item.htm?id=565725573909
获取商品: https://detail.tmall.com/item.htm?id=573621469979
获取商品: https://detail.tmall.com/item.htm?id=567834234119
啊呀
获取商品: https://detail.tmall.com/item.htm?id=568602364229
获取商品: https://detail.tmall.com/item.htm?id=527346361688
获取商品: https://detail.tmall.com/item.htm?id=545473483441
获取商品: https://detail.tmall.com/item.htm?id=566333229820
获取商品: https://detail.tmall.com/item.htm?id=571587749552
获取商品: https://detail.tmall.com/item.htm?id=570714613231
获取商品: https://detail.tmall.com/item.htm?id=530240389459
获取商品: https://detail.tmall.com/item.htm?id=538978191837
啊呀
获取商品: https://detail.tmall.com/item.htm?id=569600860699
获取商品: https://detail.tmall.com/item.htm?id=560719801867
获取商品: https://detail.tmall.com/item.htm

获取商品: https://detail.tmall.com/item.htm?id=547668154873
获取商品: https://detail.tmall.com/item.htm?id=539140592209
获取商品: https://detail.tmall.com/item.htm?id=533006210357
获取商品: https://detail.tmall.com/item.htm?id=542146971161
获取商品: https://detail.tmall.com/item.htm?id=573068450199
获取商品: https://detail.tmall.com/item.htm?id=570644964243
获取商品: https://detail.tmall.com/item.htm?id=537478295048
获取商品: https://detail.tmall.com/item.htm?id=560571281423
获取商品: https://detail.tmall.com/item.htm?id=573023695307
获取商品: https://detail.tmall.com/item.htm?id=567257664402
获取商品: https://detail.tmall.com/item.htm?id=541573389926
获取商品: https://detail.tmall.com/item.htm?id=548125275628
获取商品: https://detail.tmall.com/item.htm?id=561030552064
啊呀
啊呀
获取商品: https://detail.tmall.com/item.htm?id=572197860602
获取商品: https://detail.tmall.com/item.htm?id=568105026241
获取商品: https://detail.tmall.com/item.htm?id=531869187277
获取商品: https://detail.tmall.com/item.htm?id=565680820027
获取商品: https://detail.tmall.com/item.htm?id

289.1689887046814


In [213]:
tb = pd.concat(info_taobao,ignore_index=True)
tm = pd.concat(info_tmall,ignore_index=True)
df = pd.merge(tb,tm,how='outer')
df = df[['source','itemID','itemLink','title' ,'price','totalComment','score','good','normal', 'bad','itemDetail','shopId','shopUrl']]
df.to_csv(str('output/taobao'+target+'-info-' +str(time.strftime("%Y-%m-%d"))+'.csv'))

In [212]:
df

,source,itemID,itemLink,title,price,totalComment,score,good,normal,bad,itemDetail,shopId,shopUrl
0,taobao,574246632987,https://item.taobao.com/item.htm?id=574246632987,新款汽车行车记录仪双镜头高清夜视全景无线车载测速电子狗一体机,78.00 - 250.00,186,NaN,185.0,0.0,1.0,CCC证书编号: 主镜头光圈: F2.0品牌: 鹏之浩型号: Y806套餐: 官方标配 套餐...,64968665,xml517.taobao.com/
1,taobao,568772214439,https://item.taobao.com/item.htm?id=568772214439,高清1080P汽车行车记录仪后视镜 车载夜视一体机 单镜头 正品包邮,39.00 - 98.00,538,NaN,536.0,2.0,0.0,主镜头光圈: F2.0品牌: 中性型号: 6888套餐: 套餐一 套餐二 套餐三 套餐四安装...,66552053,shop66552053.taobao.com/
2,taobao,570805785090,https://item.taobao.com/item.htm?id=570805785090,新款汽车车载行车记录仪高清夜视全景单双镜头隐藏式电子狗一体,198.00 - 378.00,459,NaN,459.0,0.0,0.0,CCC证书编号: 主镜头光圈: F2.0品牌: 鑫诠型号: G300套餐: 官方标配 套餐一...,111271679,shop111271679.taobao.com/
3,taobao,573718312859,https://item.taobao.com/item.htm?id=573718312859,新款行车记录仪高清夜视全景360度前后双镜头录像无线24小时监控,218.00 - 348.00,459,NaN,454.0,5.0,0.0,主镜头光圈: F2.0品牌: 艾伲妠型号: AA5套餐: 官方标配安装类型: 便携式吸盘影像...,71273827,shop71273827.taobao.com/
4,taobao,534159083192,https://item.taobao.com/item.htm?id=534159083192,行车记录仪 汽车后视镜双镜头高清夜视带电子狗倒车影像一体机,122.00 - 319.00,2584,NaN,2482.0,38.0,64.0,CCC证书编号: 主镜头光圈: F2.0品牌: 鹏之浩型号: Y809套餐: 官方标配 套餐...,72578481,shop72578481.taobao.com/
5,taobao,569519448720,https://item.taobao.com/item.htm?id=569519448720,正品1080P超高清汽车行车记录仪双镜头 夜视广角迷你车载一体机,48.80 - 158.80,962,NaN,959.0,0.0,3.0,主镜头光圈: F2.0品牌: lit型号: xhs套餐: 套餐一 套餐二 套餐三 套餐四 套...,553514843,shop553514843.taobao.com/
6,taobao,571908069396,https://item.taobao.com/item.htm?id=571908069396,汽车行车记录仪单双镜头高清夜视全景电子狗后视镜倒车影像一体,198.00 - 348.00,170,NaN,170.0,0.0,0.0,CCC证书编号: 主镜头光圈: F2.0品牌: 鑫诠型号: D813套餐: 套餐一 套餐二 ...,111271679,shop111271679.taobao.com/
7,taobao,526397484286,https://item.taobao.com/item.htm?id=526397484286,领克01长城魏派wey vv5吉利博越2018款usb行车记录仪vv7专用原厂,488.00 - 758.00,2132,NaN,2127.0,3.0,2.0,CCC证书编号: 产品名称: FiTCAM/智看 HD2主镜头光圈: F1.6品牌: FiT...,145256403,shop145256403.taobao.com/
8,taobao,559369594465,https://item.taobao.com/item.htm?id=559369594465,任E行V28行车记录仪高清夜视双镜头隐藏式全景24小时停车监控WiFi,1198.00 - 1558.00,1401,NaN,1399.0,1.0,1.0,CCC证书编号: 产品名称: 任E行 V28主镜头光圈: F2.0品牌: 任E行型号: V2...,107724105,rexqiyedian.taobao.com/
9,taobao,564169946669,https://item.taobao.com/item.htm?id=564169946669,360行车记录仪G300高清夜视隐藏式迷你汽车载无线测速电子狗G300P,309.00 - 539.00,3022,NaN,2976.0,21.0,25.0,产品名称: 360 G300主镜头光圈: F2.2品牌: 360型号: G300套餐: 官方...,385022640,shop385022640.taobao.com/
